In [ ]:
# Install scikit-surprise library for recommendation system
!pip install scikit-surprise --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# ✅ Step 1: Install required packages
!pip install scikit-surprise --quiet

# ✅ Step 2: Upload the 'ratings.csv' file from the user's folder
from google.colab import files
import pandas as pd

# Prompt user to upload 'ratings.csv'
print("Please upload the 'ratings.csv' file from your local folder.")
uploaded = files.upload()  # File upload dialog will appear

# Load the uploaded ratings.csv into a DataFrame
ratings_df = pd.read_csv("ratings.csv")

# Preview the uploaded ratings data
print("\n🎬 Sample ratings (ratings.csv):")
print(ratings_df.head())

# ✅ Step 3: Upload the 'movies.csv' file from the user's folder
print("\nPlease upload the 'movies.csv' file from your local folder.")
uploaded = files.upload()  # File upload dialog will appear again

# Load the uploaded movies.csv into a DataFrame
movies_df = pd.read_csv("movies.csv")

# Preview the uploaded movies data
print("\n🎥 Sample movies (movies.csv):")
print(movies_df.head())


Please upload the 'ratings.csv' file from your local folder.


Saving ratings.csv to ratings.csv

🎬 Sample ratings (ratings.csv):
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Please upload the 'movies.csv' file from your local folder.


Saving ratings.csv to ratings (1).csv

🎥 Sample movies (movies.csv):
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [ ]:
!pip install --upgrade numpy
!pip install scikit-surprise --quiet --no-cache-dir

In [ ]:
!pip uninstall numpy
!pip install numpy

Found existing installation: numpy 2.2.5
Uninstalling numpy-2.2.5:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.2.5.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-6bb31eeb.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? pip install numpy==1.19.3
Your response ('pip install numpy==1.19.3') was not one of the expected responses: y, n, 
Proceed (Y/n)? y
  Successfully uninstalled numpy-2.2.5
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the pac

In [ ]:
!python -c "import numpy; print(numpy.__version__)"

2.2.5


In [ ]:
!pip cache purge
!pip install numpy

Files removed: 65


In [ ]:
!pip install pandas scikit-learn

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



In [ ]:
# Create a pivot table: rows = users, columns = movies, values = ratings
user_movie_matrix = ratings_df.pivot_table(index='userId', columns='movieId', values='rating')

# Fill missing values with 0
user_movie_matrix_filled = user_movie_matrix.fillna(0)

print("✅ User-Movie Matrix Shape:", user_movie_matrix_filled.shape)


✅ User-Movie Matrix Shape: (610, 9724)


In [ ]:
# Compute cosine similarity between users
user_similarity = cosine_similarity(user_movie_matrix_filled)

# Convert to DataFrame for readability
user_sim_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

print("✅ User-User Similarity Matrix Sample:")
print(user_sim_df.head())


✅ User-User Similarity Matrix Sample:
userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000  0.027283  0.059720  0.194395  0.129080  0.128152  0.158744   
2       0.027283  1.000000  0.000000  0.003726  0.016614  0.025333  0.027585   
3       0.059720  0.000000  1.000000  0.002251  0.005020  0.003936  0.000000   
4       0.194395  0.003726  0.002251  1.000000  0.128659  0.088491  0.115120   
5       0.129080  0.016614  0.005020  0.128659  1.000000  0.300349  0.108342   

userId       8         9         10   ...       601       602       603  \
userId                                ...                                 
1       0.136968  0.064263  0.016875  ...  0.080554  0.164455  0.221486   
2       0.027257  0.000000  0.067445  ...  0.202671  0.016866  0.011997   
3       0.004941  0.000000  0.000000  ...  0.005048  0.004892  0.024992   
4       0.062969  0.011361

In [ ]:
def get_movie_recommendations(user_id, n=5):
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:]  # skip self
    weighted_ratings = pd.Series(dtype=float)

    for sim_user, similarity in similar_users.items():
        sim_user_ratings = user_movie_matrix.loc[sim_user]
        weighted = sim_user_ratings * similarity
        weighted_ratings = weighted_ratings.add(weighted, fill_value=0)

    user_rated = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id].notnull()].index
    weighted_ratings = weighted_ratings.drop(user_rated, errors='ignore')

    top_movies = weighted_ratings.sort_values(ascending=False).head(n)
    return top_movies

# Example: Recommend for user 1
top_recommendations = get_movie_recommendations(user_id=1, n=5)

print("\n🎯 Top 5 Recommendations for User 1:")
for movie_id, score in top_recommendations.items():
    movie_title = movies_df.loc[movies_df['movieId'] == movie_id, 'title'].values
    title = movie_title[0] if len(movie_title) > 0 else "Unknown"
    print(f"🎬 {title}, Score: {score:.2f}")



🎯 Top 5 Recommendations for User 1:
🎬 Shawshank Redemption, The (1994), Score: 215.45
🎬 Terminator 2: Judgment Day (1991), Score: 169.40
🎬 Godfather, The (1972), Score: 150.92
🎬 Sixth Sense, The (1999), Score: 135.01
🎬 Lord of the Rings: The Fellowship of the Ring, The (2001), Score: 131.87


In [ ]:
# Optional: filter top_movies by genre
preferred_genre = 'Action'  # Or get this dynamically

# Call get_movie_recommendations to define top_movies
top_movies = get_movie_recommendations(user_id=1, n=5)

filtered = movies_df[movies_df['movieId'].isin(top_movies.index)]
filtered = filtered[filtered['genres'].str.contains(preferred_genre, case=False)]

In [ ]:
# Add release year column to movies_df
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)').astype(float)

# Optional: filter top_movies by genre
preferred_genre = 'Action'  # Or get this dynamically

# Call get_movie_recommendations to define top_movies
top_movies = get_movie_recommendations(user_id=1, n=5)

# Filter by movieId and genre from the original movies_df (which now has 'year')
filtered = movies_df[movies_df['movieId'].isin(top_movies.index)]
filtered = filtered[filtered['genres'].str.contains(preferred_genre, case=False)]

# Now filter by year using the 'year' column in filtered
filtered = filtered[filtered['year'] >= 2010]

In [ ]:
def get_movie_recommendations(user_id, n=5):
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:]  # skip self
    weighted_ratings = pd.Series(dtype=float)

    for sim_user, similarity in similar_users.items():
        sim_user_ratings = user_movie_matrix.loc[sim_user]
        weighted = sim_user_ratings * similarity
        weighted_ratings = weighted_ratings.add(weighted, fill_value=0)

    user_rated = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id].notnull()].index
    weighted_ratings = weighted_ratings.drop(user_rated, errors='ignore')

    top_movies = weighted_ratings.sort_values(ascending=False).head(n)

    # Return both top_movies and weighted_ratings
    return top_movies, weighted_ratings

# Example: Recommend for user 1
top_recommendations, weighted_ratings = get_movie_recommendations(user_id=1, n=5) # Assign the output to top_recommendations

print("\n🎯 Top 5 Recommendations for User 1:")
for movie_id, score in top_recommendations.items():
    movie_title = movies_df.loc[movies_df['movieId'] == movie_id, 'title'].values
    title = movie_title[0] if len(movie_title) > 0 else "Unknown"
    print(f"🎬 {title}, Score: {score:.2f}")


🎯 Top 5 Recommendations for User 1:
🎬 Shawshank Redemption, The (1994), Score: 215.45
🎬 Terminator 2: Judgment Day (1991), Score: 169.40
🎬 Godfather, The (1972), Score: 150.92
🎬 Sixth Sense, The (1999), Score: 135.01
🎬 Lord of the Rings: The Fellowship of the Ring, The (2001), Score: 131.87


In [ ]:
# Compute cosine similarity between movie genres (dummy encoded)
genre_matrix = movies_df['genres'].str.get_dummies(sep='|')
movie_similarity = cosine_similarity(genre_matrix)

# Average user-based and content-based scores (optional advanced step)


In [ ]:
preferred_genre = input("Enter preferred genre (e.g., Comedy): ")
min_year = int(input("Enter minimum release year: "))


Enter preferred genre (e.g., Comedy): action
Enter minimum release year: 1995


In [ ]:
def get_movie_recommendations_filtered(user_id, genre_filter="Action", min_year=1995, n=5):
    # Get similar users
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:]  # skip self
    weighted_ratings = pd.Series(dtype=float)

    for sim_user, similarity in similar_users.items():
        sim_user_ratings = user_movie_matrix.loc[sim_user]
        weighted = sim_user_ratings * similarity
        weighted_ratings = weighted_ratings.add(weighted, fill_value=0)

    # Drop movies already rated by this user
    user_rated = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id].notnull()].index
    weighted_ratings = weighted_ratings.drop(user_rated, errors='ignore')

    # Normalize score (optional)
    weighted_ratings /= similar_users.sum()

    # Merge with movie metadata
    movie_info = movies_df.copy()
    movie_info['year'] = movie_info['title'].str.extract(r'\((\d{4})\)').astype(float)

    # Apply genre and year filters
    filtered = movie_info[
        (movie_info['movieId'].isin(weighted_ratings.index)) &
        (movie_info['genres'].str.contains(genre_filter, case=False, na=False)) &
        (movie_info['year'] >= min_year)
    ]

    # Add predicted scores
    filtered['score'] = filtered['movieId'].map(weighted_ratings)

    # Get top N
    top_recs = filtered.sort_values(by='score', ascending=False).head(n)
    return top_recs[['title', 'genres', 'year', 'score']]

# 🔍 Example usage:
genre_input = "Action"
min_year_input = 1995

recommendations = get_movie_recommendations_filtered(user_id=1, genre_filter=genre_input, min_year=min_year_input)
print(f"\n🎯 Top 5 Recommendations for User 1 — Genre: {genre_input}, After {min_year_input}:\n")
print(recommendations.to_string(index=False))



🎯 Top 5 Recommendations for User 1 — Genre: Action, After 1995:

                                                        title                          genres   year    score
        Lord of the Rings: The Return of the King, The (2003)  Action|Adventure|Drama|Fantasy 2003.0 1.502235
                                    Fifth Element, The (1997)  Action|Adventure|Comedy|Sci-Fi 1997.0 1.154920
Pirates of the Caribbean: The Curse of the Black Pearl (2003) Action|Adventure|Comedy|Fantasy 2003.0 1.143482
                                      Dark Knight, The (2008)         Action|Crime|Drama|IMAX 2008.0 1.143421
                                     Kill Bill: Vol. 1 (2003)           Action|Crime|Thriller 2003.0 1.104008


<ipython-input-35-72129f49a44e>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['score'] = filtered['movieId'].map(weighted_ratings)


In [ ]:
def get_custom_recommendations(user_id, genre_filters=None, start_year=None, end_year=None, keyword=None, top_n=5):
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:]
    weighted_ratings = pd.Series(dtype=float)

    for sim_user, similarity in similar_users.items():
        sim_user_ratings = user_movie_matrix.loc[sim_user]
        weighted = sim_user_ratings * similarity
        weighted_ratings = weighted_ratings.add(weighted, fill_value=0)

    # Remove already watched
    user_rated = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id].notnull()].index
    weighted_ratings = weighted_ratings.drop(user_rated, errors='ignore')
    weighted_ratings /= similar_users.sum()  # normalize

    movie_info = movies_df.copy()
    movie_info['year'] = movie_info['title'].str.extract(r'\((\d{4})\)').astype(float)

    # Filter by genres
    if genre_filters:
        genre_filters = [g.strip().lower() for g in genre_filters]
        genre_mask = movie_info['genres'].apply(lambda g: any(gen in g.lower() for gen in genre_filters))
        movie_info = movie_info[genre_mask]

    # Filter by year range
    if start_year:
        movie_info = movie_info[movie_info['year'] >= start_year]
    if end_year:
        movie_info = movie_info[movie_info['year'] <= end_year]

    # Filter by keyword in title
    if keyword:
        movie_info = movie_info[movie_info['title'].str.contains(keyword, case=False, na=False)]

    # Merge scores
    movie_info['score'] = movie_info['movieId'].map(weighted_ratings)
    movie_info = movie_info.dropna(subset=['score'])

    # Final top recommendations
    top_recs = movie_info.sort_values('score', ascending=False).head(top_n)
    return top_recs[['title', 'genres', 'year', 'score']]


In [ ]:
def get_movie_recommendations_filtered(user_id, genre_filter="Action", min_year=1995, n=5):
    # Get similar users
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:]  # skip self
    weighted_ratings = pd.Series(dtype=float)

    for sim_user, similarity in similar_users.items():
        sim_user_ratings = user_movie_matrix.loc[sim_user]
        weighted = sim_user_ratings * similarity
        weighted_ratings = weighted_ratings.add(weighted, fill_value=0)

    # Drop movies already rated by this user
    user_rated = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id].notnull()].index
    weighted_ratings = weighted_ratings.drop(user_rated, errors='ignore')

    # Normalize score (optional)
    weighted_ratings /= similar_users.sum()

    # Merge with movie metadata
    movie_info = movies_df.copy()
    movie_info['year'] = movie_info['title'].str.extract(r'\((\d{4})\)').astype(float)

    # Apply genre and year filters
    filtered = movie_info[
        (movie_info['movieId'].isin(weighted_ratings.index)) &
        (movie_info['genres'].str.contains(genre_filter, case=False, na=False)) &
        (movie_info['year'] >= min_year)
    ]

    # Add predicted scores
    filtered['score'] = filtered['movieId'].map(weighted_ratings)

    # Get top N
    top_recs = filtered.sort_values(by='score', ascending=False).head(n)
    return top_recs[['title', 'genres', 'year', 'score']]

# 🔍 Example usage:
genre_input = "Action"
min_year_input = 1995

recommendations = get_movie_recommendations_filtered(user_id=1, genre_filter=genre_input, min_year=min_year_input)
print(f"\n🎯 Top 5 Recommendations for User 1 — Genre: {genre_input}, After {min_year_input}:\n")
print(recommendations.to_string(index=False))



🎯 Top 5 Recommendations for User 1 — Genre: Action, After 1995:

                                                        title                          genres   year    score
        Lord of the Rings: The Return of the King, The (2003)  Action|Adventure|Drama|Fantasy 2003.0 1.502235
                                    Fifth Element, The (1997)  Action|Adventure|Comedy|Sci-Fi 1997.0 1.154920
Pirates of the Caribbean: The Curse of the Black Pearl (2003) Action|Adventure|Comedy|Fantasy 2003.0 1.143482
                                      Dark Knight, The (2008)         Action|Crime|Drama|IMAX 2008.0 1.143421
                                     Kill Bill: Vol. 1 (2003)           Action|Crime|Thriller 2003.0 1.104008


<ipython-input-35-72129f49a44e>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['score'] = filtered['movieId'].map(weighted_ratings)


In [ ]:
# 🎛️ User input section
user_id = 1
genre_input = input("Enter preferred genres (comma-separated, e.g., Action,Comedy): ")
start_year = int(input("Enter start year (e.g. 1995): "))
end_year = int(input("Enter end year (e.g. 2020): "))
keyword = input("Enter keyword in title (optional): ")

# Run the recommender
genre_list = genre_input.split(',') if genre_input else []
recommendations = get_custom_recommendations(
    user_id=user_id,
    genre_filters=genre_list,
    start_year=start_year,
    end_year=end_year,
    keyword=keyword,
    top_n=5
)

print("\n🎯 Custom Top 5 Recommendations:\n")
print(recommendations.to_string(index=False))


Enter preferred genres (comma-separated, e.g., Action,Comedy): action,thriller
Enter start year (e.g. 1995): 2000
Enter end year (e.g. 2020): 2020
Enter keyword in title (optional): war

🎯 Custom Top 5 Recommendations:

                                              title                               genres   year    score
Star Wars: Episode II - Attack of the Clones (2002)         Action|Adventure|Sci-Fi|IMAX 2002.0 0.714401
Star Wars: Episode III - Revenge of the Sith (2005)              Action|Adventure|Sci-Fi 2005.0 0.646277
                           War of the Worlds (2005)     Action|Adventure|Sci-Fi|Thriller 2005.0 0.387223
  Star Wars: Episode VII - The Force Awakens (2015) Action|Adventure|Fantasy|Sci-Fi|IMAX 2015.0 0.304676
                                 Lord of War (2005)      Action|Crime|Drama|Thriller|War 2005.0 0.273162


In [ ]:
# 🎛️ User input section with Yes/No for keyword
user_id = 1
genre_input = input("Enter preferred genres (comma-separated, e.g., Action,Comedy): ")
start_year = int(input("Enter start year (e.g. 1995): "))
end_year = int(input("Enter end year (e.g. 2020): "))

use_keyword = input("Do you want to filter by a keyword in the title? (yes/no): ").strip().lower()
keyword = None
if use_keyword == 'yes':
    keyword = input("Enter keyword in title (e.g., war, love): ")


Enter preferred genres (comma-separated, e.g., Action,Comedy): action,thriller
Enter start year (e.g. 1995): 2000
Enter end year (e.g. 2020): 2020
Do you want to filter by a keyword in the title? (yes/no): yes
Enter keyword in title (e.g., war, love): war


In [ ]:
# 🎛️ User input section with Yes/No for keyword
user_id = 1
genre_input = input("Enter preferred genres (comma-separated, e.g., Action,Comedy): ")
start_year = int(input("Enter start year (e.g. 1995): "))
end_year = int(input("Enter end year (e.g. 2020): "))

use_keyword = input("Do you want to filter by a keyword in the title? (yes/no): ").strip().lower()
keyword = None
if use_keyword == 'yes':
    keyword = input("Enter keyword in title (e.g., war, love): ")

# Prepare the genre list
genre_list = genre_input.split(',') if genre_input else []

# Run the recommender with or without keyword filter
recommendations = get_custom_recommendations(
    user_id=user_id,
    genre_filters=genre_list,
    start_year=start_year,
    end_year=end_year,
    keyword=keyword,  # Will be None if "No" selected
    top_n=5
)

# Display recommendations
print("\n🎯 Custom Top 5 Recommendations:\n")
print(recommendations.to_string(index=False))
220

Enter preferred genres (comma-separated, e.g., Action,Comedy): action ,comedy
Enter start year (e.g. 1995): 1995
Enter end year (e.g. 2020): 1998
Do you want to filter by a keyword in the title? (yes/no): yes
Enter keyword in title (e.g., war, love): love

🎯 Custom Top 5 Recommendations:

                                                     title                 genres   year    score
                                Shakespeare in Love (1998)   Comedy|Drama|Romance 1998.0 0.850979
                           Everyone Says I Love You (1996) Comedy|Musical|Romance 1996.0 0.141065
                                   Addicted to Love (1997)         Comedy|Romance 1997.0 0.072563
Incredibly True Adventure of Two Girls in Love, The (1995)         Comedy|Romance 1995.0 0.022477
                         Pyromaniac's Love Story, A (1995)         Comedy|Romance 1995.0 0.012028


220

In [ ]:
# 🎛️ User input section with validation for genres and years
user_id = 1

# Load movies data (example of movie dataset being loaded)
# Make sure this is the movies dataset loaded already earlier in the process
movies_data = pd.read_csv('movies.csv')

# Extract year from title and add as a new column
movies_data['year'] = movies_data['title'].str.extract(r'\((\d{4})\)').astype(float)

# Get available genres from the dataset (combining all genres)
available_genres = set()
for genre_list in movies_data['genres'].str.split('|'):
    available_genres.update(genre_list)

# Get available years from the dataset
available_years = set(movies_data['year'])

# User inputs for genres and years
genre_input = input("Enter preferred genres (comma-separated, e.g., Action,Comedy): ").lower()
start_year = int(input("Enter start year (e.g. 1995): "))
end_year = int(input("Enter end year (e.g. 2020): "))

# Validate genres
genre_list = genre_input.split(',')
invalid_genres = [genre for genre in genre_list if genre not in available_genres]

if invalid_genres:
    print(f"\n⚠️ The following genres are not found in the dataset: {', '.join(invalid_genres)}")

# Validate years
if start_year not in available_years or end_year not in available_years:
    print(f"\n⚠️ One or both of the years are not valid in the dataset. Valid years are between {min(available_years)} and {max(available_years)}.")

# Proceed if valid genres and years
use_keyword = input("Do you want to filter by a keyword in the title? (yes/no): ").strip().lower()
keyword = None
if use_keyword == 'yes':
    keyword = input("Enter keyword in title (e.g., war, love): ")

# Run the recommender if valid input
if not invalid_genres and start_year in available_years and end_year in available_years:
    recommendations = get_custom_recommendations(
        user_id=user_id,
        genre_filters=genre_list,
        start_year=start_year,
        end_year=end_year,
        keyword=keyword,  # Will be None if "No" selected
        top_n=5
    )

    # Display recommendations
    print("\n🎯 Custom Top 5 Recommendations:\n")
    print(recommendations.to_string(index=False))
else:
    print("\n⚠️ Please fix the invalid input and try again.")

Enter preferred genres (comma-separated, e.g., Action,Comedy): cinema
Enter start year (e.g. 1995): 1990
Enter end year (e.g. 2020): 1980

⚠️ The following genres are not found in the dataset: cinema
Do you want to filter by a keyword in the title? (yes/no): no

⚠️ Please fix the invalid input and try again.
